In [1]:
!pip install neo4j

In [2]:
from datetime import datetime

from neo4j import GraphDatabase, basic_auth

In [7]:
URL = "bolt://localhost:7687"
USERNAME = "Daniel_Hurtado"
PASSWORD = "123456"
DATABASE = "taller5"

In [8]:
driver = GraphDatabase.driver(URL, auth = basic_auth(USERNAME, PASSWORD))

In [9]:
db = driver.session(database = DATABASE)

### Creating a basic graph

<img src="imgs/fig1.png">

In [10]:
summary = db.write_transaction(lambda tx: tx.run("CREATE (:Person:Owner {name: 'Pepito'})-[:OWNS]->(:Pet:Cat {name: 'Lolo'})").consume())

In [11]:
summary.counters.properties_set

2

### Attaching a new relationship to an exising node

<img src="imgs/fig2.png">

In [12]:
summary = db.write_transaction(lambda tx: tx.run("MATCH (d:Pet {name: 'Max'}) CREATE (i:Picture {createdAt:  $createdAt}) CREATE (d)-[:APPEARS_IN]->(i)", {"createdAt": datetime.today()}).consume())

In [13]:
summary.counters.properties_set

0

### Extending the network

<img src="imgs/fig3.png">

In [14]:
db.write_transaction(lambda tx: tx.run("CREATE (:Person:Owner {name: 'Marcela Reyes'})-[:OWNS]->(:Pet:Cat {name: 'Lukas'})").consume())

In [15]:
db.write_transaction(lambda tx: tx.run("MATCH (i) WHERE ID(i) = 2 MATCH (c:Pet:Cat {name: 'Lukas'}) CREATE (c)-[:APPEARS_IN]->(i)").consume())

In [16]:
db.write_transaction(lambda tx: tx.run("MATCH (c:Pet:Cat {name: 'Lukas'}) CREATE (i:Picture {createdAt:  $createdAt}) CREATE (c)-[:APPEARS_IN]->(i)", {"createdAt": datetime.today()}).consume())

### Counting nodes by type

In [17]:
result = db.read_transaction(lambda tx: tx.run("MATCH (:Pet) RETURN count(*) AS count").single())

In [18]:
result["count"]

2

### Counting number of outbound relationships from a node

In [19]:
# For a specific node
result = db.read_transaction(lambda tx: tx.run("MATCH (p:Pet) WHERE ID(p) = $node RETURN size((p)-->()) AS count", {"node": 4}).single())

In [21]:
result["count"]

TypeError: 'NoneType' object is not subscriptable

In [22]:
# For a group of nodes
result = db.read_transaction(lambda tx: list(tx.run("MATCH (p:Pet)-[r:APPEARS_IN]->(i:Picture) RETURN p.name AS name, COUNT(r) AS num_pictures")))

In [23]:
for r in result:
    print(r["name"] + " => " + str(r["num_pictures"]))

Lukas => 1


In [ ]:
db.close()